# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [2]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [3]:
context = [
    {'role': 'system', 'content': """
    CREATE TABLE employees (
        ID_usr INT PRIMARY KEY,
        name VARCHAR(100)
    );

    CREATE TABLE salary (
        ID_usr INT,
        year DATE,
        salary FLOAT,
        FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
    );

    CREATE TABLE studies (
        ID INT PRIMARY KEY,
        ID_usr INT,
        educational_level INT,
        Institution VARCHAR(255),
        Years DATE,
        Speciality VARCHAR(255),
        FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
    );
    """}
]


In [4]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
-- Maintain the SQL order simple and efficient as you can, using valid SQLite.
-- Answer the following questions using the table structures above.

-- Example 1: Get employee names and salaries for 2023
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE s.year = '2023-01-01';

-- Example 2: Find employees who studied at MIT
SELECT e.name, st.Institution
FROM employees e
JOIN studies st ON e.ID_usr = st.ID_usr
WHERE st.Institution = 'MIT';

-- Example 3: Get the highest salary among employees
SELECT MAX(salary) AS highest_salary FROM salary;

-- Example 4: Retrieve all employees who earn more than 60,000
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE s.salary > 60000;
"""})

In [5]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [6]:
#new
context_user = context.copy()
print(return_CCRMSQL("Find employees who studied at Harvard.", context_user))

SELECT e.name
FROM employees e
JOIN studies st ON e.ID_usr = st.ID_usr
WHERE st.Institution = 'Harvard';


In [7]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Find employees who studied at Harvard.", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN studies s ON e.ID_usr = s.ID_usr
WHERE s.Institution = 'Harvard';
```

This SQL query selects the names of employees who studied at Harvard by joining the "employees" table with the "studies" table on the ID_usr column and filtering the results to only include rows where the Institution is 'Harvard'.


In [8]:
#new
print(return_CCRMSQL("Get the average salary for each educational level.", context_user))

```sql
SELECT st.educational_level, AVG(s.salary) AS average_salary
FROM studies st
JOIN salary s ON st.ID_usr = s.ID_usr
GROUP BY st.educational_level;
```


In [9]:
#old
print(return_CCRMSQL("Get the average salary for each educational level.", old_context_user))

This is your SQL:
```sql
SELECT s.educational_level, AVG(s.salary) AS average_salary
FROM salary s
JOIN studies st ON s.ID_usr = st.ID_usr
GROUP BY s.educational_level;
```

This SQL query retrieves the average salary for each educational level by joining the "salary" table with the "studies" table on the user ID. It then calculates the average salary for each educational level group using the AVG function and groups the results by educational level.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [10]:
NL_queries = [
    "Get the names of all employees."
    "Find the salary of employee Bob Smith."
    "List all employees sorted by name in ascending order."
    "Count how many employees are in the database."
    "Find all employees who started studying after 2018."
]

In [12]:
#new
context_user = context.copy()
print(return_CCRMSQL("Get the names of all employees.", context_user))

SELECT name FROM employees;


In [13]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Get the names of all employees.", old_context_user))

This is your SQL:
```sql
SELECT name FROM employees;
```
This SQL command selects the "name" column from the "employees" table, retrieving the names of all employees.


In [14]:
#new
print(return_CCRMSQL("Find all employees who started studying after 2018.", context_user))

SELECT e.name
FROM employees e
JOIN studies st ON e.ID_usr = st.ID_usr
WHERE st.Years > '2018-01-01';


In [15]:
#old
print(return_CCRMSQL("Find all employees who started studying after 2018.", old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN studies s ON e.ID_usr = s.ID_usr
WHERE s.Years > '2018-01-01';
```

This SQL query selects the names of employees who started studying after 2018 by joining the "employees" table with the "studies" table on the ID_usr column and filtering the results to only include records where the Years column in the "studies" table is after January 1, 2018.


In [16]:
#new
print(return_CCRMSQL("Count how many employees are in the database.", context_user))

```sql
SELECT COUNT(*) AS total_employees
FROM employees;
```


In [17]:
#old
print(return_CCRMSQL("Count how many employees are in the database.", old_context_user))

This is your SQL:
```sql
SELECT COUNT(*) AS total_employees
FROM employees;
```
This SQL query counts the total number of employees in the database by selecting all records from the employees table and returning the count.


- Findings:
    The new prompt produced more structured, efficient, and readable SQL queries compared to the old prompt. Both prompts generated mostly correct SQL.
    
    The new prompt improved readability and maintained a cleaner format.

- Lessons Learned:
    Providing Few-Shot Examples improves SQL accuracy and reduces mistakes.